In [1]:
from master3 import *

plt_backend
PID: 14408


In [2]:
from first import *


In [4]:
# Read video
fnames = sorted(
    glob.glob(
        "/Volumes/easystore/data_visss/master_data/mosaic/visss1_visss_master_S1145792/data/2019/11/15/visss1_visss_master_S1145792_20191115-05*.mov"
    ))
fnames = sorted(
    glob.glob(
        "/Volumes/easystore/data_visss/slave_data/mosaic/visss2_visss_slave_S1143155/data/2019/11/15/visss2_visss_slave_S1143155_20191115-05*.mov"
    ))

# fnames = sorted(glob.glob(
#     "/Volumes/easystore/data_visss/slave_data/mosaic/visss2_visss_slave_S1143155/data/2019/11/15/visss2_visss_slave_S1143155_20191115-05*.mov"))

# fnames = sorted(glob.glob(
#     '/Users/mmaahn/data/VISSS_data/visss1_visss_master_S1145792_20191115-052500.mov'))
# fnames = sorted(glob.glob(
#     '/Users/mmaahn/data/VISSS_data/visss2_visss_slave*.mov'))

outPath = '/Users/mmaahn/data/VISSS_data//mosaic_retrieval'

for fname in fnames:
    print(fname)
    fnameOut = '%s/%s.' % (outPath, fname.split('/')[-1])
    fnameOut2 = '%s/%s.retrieval_v%s.nc' % (outPath, fname.split('/')[-1],
                                            version)
    particles1 = particles(fnameOut, verbosity=0)

    outFname = '%s/%s.nc' % (outPath, fname.split('/')[-1])
    if os.path.isfile(fnameOut2):
        print(outFname, 'exists')
        continue

    inVid = cv2.VideoCapture(fname)

    ii = -1
    if not inVid.isOpened:
        print('Unable to open: ')
        exit(0)

    blurs = np.zeros(int(inVid.get(cv2.CAP_PROP_FRAME_COUNT)))
    nMovingPixs = np.zeros(int(inVid.get(cv2.CAP_PROP_FRAME_COUNT)))

    while True:

        ii += 1

        if ii % 100 == 0:
            print(fname, ii)

        # if ii < 5825: continue

        # inVid.set(cv2.CAP_PROP_POS_FRAMES,5825);

        ok, fullFrame = inVid.read()
        if fullFrame is None:
            break

        # frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        frame = fullFrame[64:]

        particles1.update(frame)

        #print("FID", particles1.fid, particles1.nMovingPix, particles1.blur, particles1.N)

#         if particles1.N > 0:

#             cv2.imshow('Frame', particles1.frame4drawing)
#             cv2.imshow('FG Mask', particles1.fgMask)
#             keyboard = cv2.waitKey(10)
#             if keyboard == 'q' or keyboard == 27:
#                 break

    res = particles1.collectResults()
    res.to_netcdf(fnameOut2)
    # input("Press Enter to continue...")
    # else:
    #     # print('skip', nMovingPix, blur)
    #     pass
    # # outVid.write(frame)

    # outVid.close()
    # inVid.close()

    # results = xr.Dataset({
    #     'blur': xr.DataArray(blurs, dims=['frame']),
    #     'nMovingPixs': xr.DataArray(nMovingPixs, dims=['frame']),
    #     })
    # results.to_netcdf(outFname)